## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
# mlflow.list_experiments()   # Deprecated
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1727432846678, experiment_id='0', last_update_time=1727432846678, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/09/27 10:29:58 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2024/09/27 10:30:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/27 10:30:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run learned-lynx-140 at: http://127.0.0.1:5000/#/experiments/1/runs/4a23465d61ef46fa98bb67e44a937565.
2024/09/27 10:30:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


default artifacts URI: 'mlflow-artifacts:/1/4a23465d61ef46fa98bb67e44a937565/artifacts'


In [7]:
# mlflow.list_experiments()   # Deprecated
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1727356398072, experiment_id='1', last_update_time=1727356398072, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1727356268605, experiment_id='0', last_update_time=1727356268605, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [8]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [10]:
# client.list_registered_models()
client.search_registered_models()

[]

In [20]:
# run_id = client.list_run_infos(experiment_id='1')[0].run_id
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/09/26 13:19:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1727356748651, current_stage='None', description='', last_updated_timestamp=1727356748651, name='iris-classifier', run_id='dd64880183f14f63ab9ff4c2c81fb370', run_link='', source='mlflow-artifacts:/1/dd64880183f14f63ab9ff4c2c81fb370/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [21]:
# client.list_registered_models()
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1727356748612, description='', last_updated_timestamp=1727356748651, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1727356748651, current_stage='None', description='', last_updated_timestamp=1727356748651, name='iris-classifier', run_id='dd64880183f14f63ab9ff4c2c81fb370', run_link='', source='mlflow-artifacts:/1/dd64880183f14f63ab9ff4c2c81fb370/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]